topic_gate(llamaindexを用いたdb検索の結果をプロンプトに挿入するかを判断する言語モデル。)

In [103]:
dialogue_list = []

with open("topicgate用.txt","r") as f:
    row = []
    for line in f:
        row = line.split("\t")
        row[1] = row[1][:-1]
        dialogue_list.append(row)

pythonで作る対話システム配布コーパスを別会話ごとに区切る

In [105]:
splited_dialogue_list = []

dialogues = []
for i in range(1,len(dialogue_list)-1):
    if (dialogue_list[i-1][1] != dialogue_list[i][0]):
        splited_dialogue_list.append(dialogues)
        dialogues = []
    else:
        dialogues.append(dialogue_list[i-1])

In [185]:
with open("dialogue_splited_by_topic.txt","w") as f:
    for row1 in splited_dialogue_list:
        for row2 in row1:
            f.write(f"{row2[0]}\t{row2[1]}\n")
        f.write("\n\n")

topicラベルが付与されたコーパスから訓練用データを作成。

you will be shown 1 Query Sentence and SOME Topic Options. Please read, memorize, and understand given materials, then complete the task under the guidance of Task Introduction.

Query Sentence:

Topic Options:

Latest Topic:


From the Topic choices, select one or more topics that are relevant to the Query Sentence. Latest Topic is the topic of the most recent conversation. If the conversations are consecutive, it inherits the topic of the most recent conversation. If the appropriate topic does not exist in the topic options, create a new topic and select the NOTO Option. The contents of the options are not reported except for the NOTO option, but only the number of the selected option as a '#' delimited string. For example, if topic options N and M are selected, the output would be: #N#M. However, when NOTO option is selected, the new label you created should be output instead of the option number. That is, the output will look like this: #N#M#0:NEW_TOPIC task Query Sentence, the number of the selected option must be greater than 0 and must not exceed the total number of topic options NUM.

In [234]:
prompt = """
you will be shown 1 Query Sentence, SOME Topic Options, Latest Topic, Conversation History. Please read, memorize, and understand given materials, then complete the task under the guidance of Task Introduction.

Query Sentence:

Topic Options:

Latest Topic:

Conversation History:


From the Topic choices, select one or more topics that are relevant to Query Sentence and Conversation History. Latest Topic is the topic of the most recent conversation. If the conversations are consecutive, it inherits the topic of the most recent conversation. If the appropriate topic does not exist in the topic options, create a new topic and select the NOTO Option. The contents of the options are not reported except for the NOTO option, but only the number of the selected option as a '#' delimited string. For example, if topic options N and M are selected, the output would be: #N#M. However, when NOTO option is selected, the new label you created should be output instead of the option number. That is, the output will look like this: #N#M#0:NEW_TOPIC task Query Sentence, the number of the selected option must be greater than 0 and must not exceed the total number of topic options NUM.
"""

In [238]:
import ast

dialogue_list = []

with open("dialogue_splited_by_topic_hold_label.txt","r") as f:
    for line in f:
        parts = line[:-1].split(",", 2)
        if (len(parts) == 3):
            topic_list = parts[2]
            topic_list = topic_list[1:-1].strip()
            topic_list = topic_list.split(",")
            parts[2] = topic_list
            dialogue_list.append(parts)
        else:
            dialogue_list.append([])

In [202]:
print(dialogue_list)

[['最近はまっていることなどはありますか？', '朝ドラでしょうか。', ['趣味']], ['どんなタイトルなんですか？', 'スカーレット、です。', ['趣味', 'ドラマ']], ['みどころはどのあたりですか？', '笑えるところです。劇中の会話が漫才みたいな時があります。大阪制作らしいドラマです。', ['趣味', ' ドラマ']], ['笑えるのは良いですね、感動や悲劇などもあるのでしょうか？', '時々あります。', ['趣味', ' ドラマ']], ['70年くらい前の時代設定なんでしたっけ', '現時点の放送では、昭和34年という設定です。', ['趣味', ' ドラマ']], ['激動の時代ですね。タイトルの由来が気になります', '緋色、赤という意味で、女性初の陶芸家ということで、陶器を焼く炎と女性ということをあらわしてたと思います。', ['趣味', ' ドラマ']], ['とても楽しそうな話ですね、興味が湧いてきました', '近々、ヒロインが結婚するような気配があります。', ['趣味', ' ドラマ']], ['それはひとつの佳境ですね、ヒロインは戸田エリカさんでしたっけ？', 'そうです。でも戸田さんは恵梨香で、エリカは今問題になってる人ですね。', ['趣味', ' ドラマ', ' 女優']], ['そうでしたね！どちらも素敵な女優さんなのに残念です', '確か、今日昼の情報番組でやってたと思うんですけど、大河ドラマ降板が決まったようです。', ['趣味', ' ドラマ', ' 女優']], [], ['こんばんは、よろしくお願いいたします', 'こんばんは、こちらこそよろしくお願いいたします！', ['挨拶']], ['昨日、今日と暖かい気候でしたね。どこかへおでかけされましたか？', 'はい、昨日は友人たちと、市内で開催されているイベントに行き、アクセサリーなどを購入しました。', ['挨拶', ' 天気']], ['楽しそうなイベントですね、どんなお店が出ていましたか？', '全国各地で手作りのアクセサリーや衣類、陶器などを売っている人たちが集まっていました。素敵なものばかりでした。', ['挨拶', ' 天気', ' 趣味', ' イベント']], ['そんなイベントがあるんですね、私も行きたかったです', 'お近くでそう

In [248]:
topic_option = {"(0)NOTO.None of the others":0}
traning_data_jsons = []

# 新しいtopic作成時のID付与に使用する
count = 1
# 各行で選択されているtopicのidが代入されるリスト
selected_option_id_list = []
# 上リスト内に入るIDが一時的に代入される
selected_option_id = 0
# NOTOが選択された際にその内容を格納する
NOTO_topic_list = []
# NOTOが選択された際にその内容を新しいトピックに登録してIDを格納する
NOTO_topic_id_list = []
# user_input作成時に挿入するtopic_option
topic_option_input = ""
# 言語モデルの応答
response = ""
# ひとつ前の会話の言語モデルの応答を格納。Latest Topicを作成するため。
pre_response = ""
# 会話が途切れるまでの会話を記録
conversation_history = ""
# チャットボットが直前に作った回答を保存
pre_chat_bot_answer = ""
# 現在の会話のトピックID
latest_topic = ""
# 直近の会話のトピックID
pre_latest_topic = ""

for row in dialogue_list:
    json_elm = {}
    selected_option_id_list = []
    NOTO_topic_list = []
    NOTO_topic_id_list = []
    if len(row) == 3:
        for hold_topic in row[2]:
            hold_topic = hold_topic.strip()
            topic_list = [key for key,value in topic_option.items()]
            if (hold_topic not in topic_list):
                NOTO_topic_list.append(hold_topic)
                topic_option[hold_topic] = count
                NOTO_topic_id_list.append(count)
                count += 1
                selected_option_id = 0
                # print(hold_topic)
            else:
                # print("1")
                selected_option_id = topic_option[hold_topic]
            selected_option_id_list.append(selected_option_id)
        topic_option_input = ""
        for key,value in topic_option.items():
            if key not in NOTO_topic_list:
                one_topic = f"({value}):{key}"
                topic_option_input = f"{topic_option_input}{one_topic}\n"
        response = ""
        NOTO_count = 0
        latest_topic = ""
        for topic_elm in selected_option_id_list:
            if (topic_elm != 0):
                response += f"#{topic_elm}"
                latest_topic += f"#{topic_elm}"
            else:
                response += f"#0:{NOTO_topic_list[NOTO_count]}"
                latest_topic += f"#{NOTO_topic_id_list[NOTO_count]}"
                NOTO_count += 1
        # print(topic_option_input)
        query = f"""
Query Sentence:
{pre_chat_bot_answer},{row[0]}

Topic Options:
{topic_option_input}

Latest Topic:
{pre_latest_topic}

Conversation History:
{conversation_history}
    """
        json_elm = {"messages": [{"role": "system", "content": prompt}, {"role": "user", "content":query}, {"role": "assistant", "content": response}]}
        traning_data_jsons.append(json_elm)
        pre_response = response
        pre_latest_topic = latest_topic
        conversation_history += f"{row[0]},{row[1]}\n"
        pre_chat_bot_answer = row[1]
    else:
        conversation_history = ""
    

In [249]:
import json

with open(r"gpt_train_jsonl\traning_data_for_topic_gate.jsonl","w") as f:
    for row in traning_data_jsons:
        # print(row)
        json.dump(row,f,ensure_ascii=False)
        f.write("\n")
